In [67]:
!pip install human-eval

In [85]:
from human_eval.data import read_problems
sample_problem = read_problems()['HumanEval/84']

In [86]:
sample_problem['test']

'def check(candidate):\n\n    # Check some simple cases\n    assert True, "This prints if this assert fails 1 (good for debugging!)"\n    assert candidate(1000) == "1", "Error"\n    assert candidate(150) == "110", "Error"\n    assert candidate(147) == "1100", "Error"\n\n    # Check some edge cases that are easy to work out by hand.\n    assert True, "This prints if this assert fails 2 (also good for debugging!)"\n    assert candidate(333) == "1001", "Error"\n    assert candidate(963) == "10010", "Error"\n\n'

In [87]:
#!/usr/bin/env python3
"""
Extract assert test cases from selected HumanEval problems
and save them as a JSON dictionary.
"""

import re
import json
from human_eval.data import read_problems

def extract_asserts(task_ids):
    """
    Extract all assert lines for given task IDs.
    Returns a dict: {task_id: [assert lines]}.
    """
    problems = read_problems()
    extracted = {}

    for tid in task_ids:
        if tid not in problems:
            extracted[tid] = ["Task ID not found."]
            continue

        test_code = problems[tid]["test"]
        # Extract only assert lines
        asserts = re.findall(r'assert .+', test_code)
        extracted[tid] = asserts

    return extracted


def write_to_json(test_cases, filename="test_cases.json"):
    """
    Write extracted test cases to a JSON file.
    """
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(test_cases, f, indent=4, ensure_ascii=False)
    print(f"✅ Test cases written to {filename}")


if __name__ == "__main__":
    # Example usage
    task_list = [
    "HumanEval/1", "HumanEval/17", "HumanEval/26", "HumanEval/77",
    "HumanEval/84", "HumanEval/94", "HumanEval/101", "HumanEval/107",
    "HumanEval/114", "HumanEval/156"
]
    test_cases = extract_asserts(task_list)
    write_to_json(test_cases, "humaneval_testcases.json")


✅ Test cases written to humaneval_testcases.json


In [84]:
def solve(N):
    """Given a positive integer N, return the total sum of its decimal digits in binary.

    Example:
        For N = 1000, the sum of digits will be 1, the output should be "1".
        For N = 150, the sum of digits will be 6, the output should be "110".
        For N = 147, the sum of digits will be 12, the output should be "1100".

    Variables:
        @N integer
             Constraints: 0 ≤ N ≤ 10000.
    Output:
         a string of binary number
    """
    digit_sum = sum(int(digit) for digit in str(N))  # Sum of decimal digits
    return bin(digit_sum)[2:]  # Convert sum to binary and remove '0b'
solve(147)

'1100'

In [26]:
print(sample_problem)
print("test")
print(sample_problem['test'])
print("*"*100)
import pandas as pd

# Path to your Excel file
file_path = "/content/planning_gemma.xlsx"

# Read the Excel file
df = pd.read_excel(file_path)

# Display the DataFram
print("/n/n")
print("code ")
print(df['extracted_code'][0])


{'task_id': 'HumanEval/1', 'prompt': 'from typing import List\n\n\ndef separate_paren_groups(paren_string: str) -> List[str]:\n    """ Input to this function is a string containing multiple groups of nested parentheses. Your goal is to\n    separate those group into separate strings and return the list of those.\n    Separate groups are balanced (each open brace is properly closed) and not nested within each other\n    Ignore any spaces in the input string.\n    >>> separate_paren_groups(\'( ) (( )) (( )( ))\')\n    [\'()\', \'(())\', \'(()())\']\n    """\n', 'entry_point': 'separate_paren_groups', 'canonical_solution': "    result = []\n    current_string = []\n    current_depth = 0\n\n    for c in paren_string:\n        if c == '(':\n            current_depth += 1\n            current_string.append(c)\n        elif c == ')':\n            current_depth -= 1\n            current_string.append(c)\n\n            if current_depth == 0:\n                result.append(''.join(current_string

{'error': "Execution failed: type complex doesn't define __round__ method",
 'traceback': 'Traceback (most recent call last):\n  File "/tmp/ipython-input-2309236678.py", line 35, in evaluate_generated_code\n    check_func(candidate_func)\n  File "<string>", line 6, in check\n  File "<string>", line 16, in iscube\nTypeError: type complex doesn\'t define __round__ method\n'}

In [66]:
output_path = "/content/eval_results_gemma.xlsx"  # change path as needed
new_df.to_excel(output_path, index=False)



In [59]:
import traceback
import ast

def evaluate_generated_code(problem_dict, generated_code: str):
    """
    Evaluate generated code against HumanEval-style tests.
    Always runs *all* asserts individually, capturing input, expected, actual outputs.

    Args:
        problem_dict (dict): HumanEval-like dict with keys:
            - 'entry_point'
            - 'test'
        generated_code (str): Model-generated code string

    Returns:
        dict: {'results': list of per-test dictionaries}
    """
    entry_point = problem_dict['entry_point']
    test_code = problem_dict['test']
    results = []

    try:
        # Compile and load generated code
        namespace = {}
        exec(generated_code, namespace)
        if entry_point not in namespace:
            return {"error": f"Entry point '{entry_point}' not found in generated code."}
        candidate_func = namespace[entry_point]
    except Exception as e:
        return {"error": f"Failed to execute generated code: {e}", "traceback": traceback.format_exc()}

    # Extract assert statements only
    test_lines = [l.strip() for l in test_code.splitlines() if l.strip().startswith("assert ")]
    if not test_lines:
        return {"error": "No assert statements found in test code."}

    for line in test_lines:
        try:
            expr = line.replace("assert ", "").strip()
            if "==" not in expr:
                continue

            left, right = expr.split("==", 1)
            left = left.strip()
            right = right.strip()

            # Extract the candidate call and its arguments
            input_part = left[left.find("(")+1:left.rfind(")")]
            expected_output = ast.literal_eval(right)

            # Evaluate actual output safely
            try:
                actual_output = eval(f"candidate({input_part})", {"candidate": candidate_func})
                passed = actual_output == expected_output
                results.append({
                    "test_case": expr,
                    "input": input_part,
                    "expected": expected_output,
                    "actual": actual_output,
                    "status": "✅ Passed" if passed else "❌ Failed"
                })
            except Exception as e:
                results.append({
                    "test_case": expr,
                    "input": input_part,
                    "expected": expected_output,
                    "actual": str(e),
                    "status": f"❌ Runtime Error ({type(e).__name__})"
                })

        except Exception as e:
            results.append({
                "test_case": line,
                "error": f"Failed to parse test line: {e}",
                "status": "⚠️ Parse Error"
            })

    return {"results": results}
generated_code = df['extracted_code'][21]  # or any generated code
sample_problem = read_problems()[df['task_id'][21]]

results = evaluate_generated_code(sample_problem, generated_code)


In [60]:
results

{'results': [{'test_case': 'candidate(123) == (8, 13)',
   'input': '123',
   'expected': (8, 13),
   'actual': (8, 13),
   'status': '✅ Passed'},
  {'test_case': 'candidate(12) == (4, 6)',
   'input': '12',
   'expected': (4, 6),
   'actual': (4, 6),
   'status': '✅ Passed'},
  {'test_case': 'candidate(3) == (1, 2)',
   'input': '3',
   'expected': (1, 2),
   'actual': (1, 2),
   'status': '✅ Passed'},
  {'test_case': 'candidate(63) == (6, 8)',
   'input': '63',
   'expected': (6, 8),
   'actual': (6, 8),
   'status': '✅ Passed'},
  {'test_case': 'candidate(25) == (5, 6)',
   'input': '25',
   'expected': (5, 6),
   'actual': (5, 6),
   'status': '✅ Passed'},
  {'test_case': 'candidate(19) == (4, 6)',
   'input': '19',
   'expected': (4, 6),
   'actual': (4, 6),
   'status': '✅ Passed'},
  {'test_case': 'candidate(9) == (4, 5), "This prints if this fails 1 (good for debugging!)"',
   'input': '9',
   'expected': ((4, 5), 'This prints if this fails 1 (good for debugging!)'),
   'actual

In [38]:
new_df=df.copy()
new_df['output_description']=['']*len(df)
for ind in df.index:
  sample_problem = read_problems()[df['task_id'][ind]]

  new_df['output_description'][ind]=evaluate_generated_code(sample_problem,df['extracted_code'][ind])


/tmp/ipython-input-1421575472.py:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  new_df['output_description'][ind]=evaluate_generated_code(sample_problem,df['extracted_code'][ind])
/tmp/ipython-input-1421575472.py:6: SettingWithCopyWarning:

In [39]:
new_df

,task_id,prompt,model_output_index,model_output,extracted_code,result,passed,pass@1,pass@3,output_description
0,HumanEval/1,You are an expert Python programmer. Given the...,1,```python\nfrom typing import List\n\n\ndef se...,from typing import List\n\n\ndef separate_pare...,"{""task_id"": ""HumanEval/1"", ""passed"": true, ""re...",True,1,1,"{'results': [('All tests', '✅ Passed')]}"
1,HumanEval/1,You are an expert Python programmer. Given the...,2,```python\nfrom typing import List\n\n\ndef se...,from typing import List\n\n\ndef separate_pare...,"{""task_id"": ""HumanEval/1"", ""passed"": true, ""re...",True,1,1,"{'results': [('All tests', '✅ Passed')]}"
2,HumanEval/1,You are an expert Python programmer. Given the...,3,```python\nfrom typing import List\n\n\ndef se...,from typing import List\n\n\ndef separate_pare...,"{""task_id"": ""HumanEval/1"", ""passed"": true, ""re...",True,1,1,"{'results': [('All tests', '✅ Passed')]}"
3,HumanEval/17,You are an expert Python programmer. Given the...,1,```python\nfrom typing import List\n\n\ndef pa...,from typing import List\n\n\ndef parse_music(m...,"{""task_id"": ""HumanEval/17"", ""passed"": true, ""r...",True,1,1,"{'results': [('All tests', '✅ Passed')]}"
4,HumanEval/17,You are an expert Python programmer. Given the...,2,```python\nfrom typing import List\n\n\ndef pa...,from typing import List\n\n\ndef parse_music(m...,"{""task_id"": ""HumanEval/17"", ""passed"": true, ""r...",True,1,1,"{'results': [('All tests', '✅ Passed')]}"
5,HumanEval/17,You are an expert Python programmer. Given the...,3,```python\nfrom typing import List\n\n\ndef pa...,from typing import List\n\n\ndef parse_music(m...,"{""task_id"": ""HumanEval/17"", ""passed"": true, ""r...",True,1,1,"{'results': [('All tests', '✅ Passed')]}"
6,HumanEval/26,You are an expert Python programmer. Given the...,1,```python\nfrom typing import List\n\n\ndef re...,from typing import List\n\n\ndef remove_duplic...,"{""task_id"": ""HumanEval/26"", ""passed"": true, ""r...",True,1,1,"{'results': [('All tests', '✅ Passed')]}"
7,HumanEval/26,You are an expert Python programmer. Given the...,2,```python\nfrom typing import List\n\n\ndef re...,from typing import List\n\n\ndef remove_duplic...,"{""task_id"": ""HumanEval/26"", ""passed"": true, ""r...",True,1,1,"{'results': [('All tests', '✅ Passed')]}"
8,HumanEval/26,You are an expert Python programmer. Given the...,3,```python\nfrom typing import List\n\n\ndef re...,from typing import List\n\n\ndef remove_duplic...,"{""task_id"": ""HumanEval/26"", ""passed"": true, ""r...",True,1,1,"{'results': [('All tests', '✅ Passed')]}"
9,HumanEval/77,You are an expert Python programmer. Given the...,1,```python\ndef iscube(a):\n '''\n Write ...,def iscube(a):\n '''\n Write a function ...,"{""task_id"": ""HumanEval/77"", ""passed"": false, ""...",False,0,0,{'error': 'Execution failed: type complex does...


In [29]:
print(df['extracted_code'][11])

def iscube(a):
    '''
    Write a function that takes an integer a and returns True 
    if this ingeger is a cube of some integer number.
    Note: you may assume the input is always valid.
    Examples:
    iscube(1) ==> True
    iscube(2) ==> False
    iscube(-1) ==> True
    iscube(64) ==> True
    iscube(0) ==> True
    iscube(180) ==> False
    '''
    # Calculate the cube root of the input number.
    # If the cube of the cube root is equal to the original number, then it is a cube.
    cube_root = round(a**(1/3))
    return cube_root**3 == a
